In [24]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [25]:
# dimensions of our images.
img_width, img_height = 150, 150

In [35]:
import os
#os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\dogs-vs-cats\\')
os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\dogs-vs-cats')
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'train\\train'
validation_data_dir = 'validation\\validation'
nb_train_samples = 102
nb_validation_samples = 40
epochs = 2
batch_size = 1

In [36]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator,102)
    
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(generator,40)
    
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    


In [37]:
save_bottlebeck_features()

Found 602 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [38]:
train_data = np.load(open('bottleneck_features_train.npy',mode='rb'))
train_data.shape

(102, 4, 4, 512)

In [39]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy',mode='rb'))
    train_labels = np.array(
        [0] * int((nb_train_samples / 2)) + [1] * int((nb_train_samples / 2)))

    validation_data = np.load(open('bottleneck_features_validation.npy',mode='rb'))
    validation_labels = np.array(
        [0] * int((nb_validation_samples / 2)) + [1] * int((nb_validation_samples / 2)))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    
    score = model.evaluate(train_data, train_labels, verbose=0) 
    print('Train score:', score[0]) 
    print('Train accuracy:', score[1])
    score = model.evaluate(validation_data, validation_labels, verbose=0) 
    print('Test score:', score[0]) 
    print('Test accuracy:', score[1])
    
    model.save_weights(top_model_weights_path)

In [40]:
train_top_model()

Instructions for updating:
Use tf.cast instead.
Train on 102 samples, validate on 40 samples
Epoch 1/2
102/102 [==============================] - 4s 39ms/step - loss: 2.7833 - acc: 0.3922 - val_loss: 0.9762 - val_acc: 0.4750
Epoch 2/2
102/102 [==============================] - 3s 34ms/step - loss: 1.2838 - acc: 0.6471 - val_loss: 0.9071 - val_acc: 0.5500
Train score: 0.45166317680302787
Train accuracy: 0.7745098027528501
Test score: 0.9070775508880615
Test accuracy: 0.55
